## Assignment 2: Using Collaborative Filtering Method For the Assigned Project

### Vineeth Marikuntematha Ravisharadhya a1896845

### Importing the necessary libraries

In [1]:
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. Read the dataset
#### I have used dataset basket_data_by_date_train_big.csv and basket_data_by_date_test_big.csv

In [3]:
# Load the training dataset
train_data = pd.read_csv("basket_data_by_date_train_big.csv")

# Load the test dataset
test_data = pd.read_csv("basket_data_by_date_test_big.csv")

### 2.Exploratory Data Analysis


1.   Checking the first few rows
2.   Checking the are there any null values





In [4]:
train_data.head()


,BillNo,Itemname,Quantity,Date,Price,CustomerID,cost
0,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,1/12/2010 8:26,3.39,17850,20.34
1,536365,GLASS STAR FROSTED T-LIGHT HOLDER,6,1/12/2010 8:26,4.25,17850,25.50
2,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,1/12/2010 8:26,2.55,17850,15.30
3,536365,RED WOOLLY HOTTIE WHITE HEART.,6,1/12/2010 8:26,3.39,17850,20.34
4,536365,SET 7 BABUSHKA NESTING BOXES,2,1/12/2010 8:26,7.65,17850,15.30


In [5]:
test_data.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,cost
0,572741,CHRISTMAS CRAFT LITTLE FRIENDS,6,25/10/2011 15:47,2.10,14341,12.60
1,572741,BISCUIT TIN 50'S CHRISTMAS,6,25/10/2011 15:47,2.89,14341,17.34
2,572741,CHRISTMAS CRAFT TREE TOP ANGEL,6,25/10/2011 15:47,2.10,14341,12.60
3,572741,ROTATING SILVER ANGELS T-LIGHT HLDR,6,25/10/2011 15:47,2.55,14341,15.30
4,572741,PAPER CHAIN KIT 50'S CHRISTMAS,6,25/10/2011 15:47,2.95,14341,17.70


In [6]:

train_data.shape



(200000, 7)

In [7]:
test_data.shape

(65326, 7)

In [8]:

train_data.describe()



,BillNo,Quantity,Price,CustomerID,cost
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,555444.533915,3.720960,3.609832,15461.086845,11.227514
std,10778.632198,2.763365,4.928895,1687.747367,11.995236
min,536365.000000,1.000000,0.060000,12347.000000,0.100000
25%,546084.000000,1.000000,1.650000,14096.000000,3.300000
50%,555653.000000,3.000000,2.550000,15464.000000,8.400000
75%,565324.000000,6.000000,4.250000,16931.000000,16.500000
max,572741.000000,10.000000,295.000000,18287.000000,527.700000


In [9]:

test_data.describe()

,BillNo,Quantity,Price,CustomerID,cost
count,65326.000000,65326.000000,65326.000000,65326.000000,65326.000000
mean,577106.526743,3.354116,3.144051,15453.535300,9.233652
std,2497.493577,2.634817,3.423463,1681.188085,10.541928
min,572741.000000,1.000000,0.060000,12347.000000,0.060000
25%,574898.000000,1.000000,1.250000,14096.000000,2.500000
50%,577086.500000,2.000000,2.080000,15453.000000,5.850000
75%,579142.000000,5.000000,3.950000,16929.000000,14.850000
max,581587.000000,10.000000,165.000000,18283.000000,510.000000


In [10]:
train_data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   BillNo      200000 non-null  int64  
 1   Itemname    200000 non-null  object 
 2   Quantity    200000 non-null  int64  
 3   Date        200000 non-null  object 
 4   Price       200000 non-null  float64
 5   CustomerID  200000 non-null  int64  
 6   cost        200000 non-null  float64
dtypes: float64(2), int64(3), object(2)
memory usage: 10.7+ MB


In [11]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65326 entries, 0 to 65325
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   BillNo      65326 non-null  int64  
 1   Itemname    65326 non-null  object 
 2   Quantity    65326 non-null  int64  
 3   Date        65326 non-null  object 
 4   Price       65326 non-null  float64
 5   CustomerID  65326 non-null  int64  
 6   cost        65326 non-null  float64
dtypes: float64(2), int64(3), object(2)
memory usage: 3.5+ MB


In [12]:
train_data.isnull().sum()

BillNo        0
Itemname      0
Quantity      0
Date          0
Price         0
CustomerID    0
cost          0
dtype: int64

In [13]:
test_data.isnull().sum()

BillNo        0
Itemname      0
Quantity      0
Date          0
Price         0
CustomerID    0
cost          0
dtype: int64

### 3.Data Preprocessing And Feature Engineering

#### I have used the required libraries and also have used scipy sparse matrix to convert the dataframe that has been created

In [14]:
# Required Libraries
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from scipy.sparse import csr_matrix
import random

In [15]:
# Modifying the train_data to make it compatible with the recommendation system
train_data_modified = train_data.groupby(['BillNo', 'Itemname', 'CustomerID']).agg({'Quantity': 'sum'}).reset_index()

# Aggregate the data to ensure each combination of Itemname and CustomerID is unique
train_data_modified_agg = train_data_modified.groupby(['Itemname', 'CustomerID']).agg({'Quantity': 'sum'}).reset_index()

# Pivot the train_data_modified_agg to make it suitable for recommendation system
pivot_train_data = train_data_modified_agg.pivot(index='Itemname', columns='CustomerID', values='Quantity').fillna(0)

# Convert dataframe of movie features to scipy sparse matrix
csr_data = csr_matrix(pivot_train_data.values)

### 4.Data Modeling

In [16]:

# Define model
knn = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=20, n_jobs=-1)

# Fit
knn.fit(csr_data)

def get_item_recommendation(item_name):
    n_items_to_recommend = 10
    if item_name in pivot_train_data.index:
        item_idx = pivot_train_data.index.get_loc(item_name)
        distances, indices = knn.kneighbors(csr_data[item_idx].reshape(1, -1), n_neighbors=n_items_to_recommend+1)
        rec_item_indices = sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]

        recommend_frame = []
        for val in rec_item_indices:
            idx = pivot_train_data.index[val[0]]
            recommend_frame.append({'ItemName': idx, 'Distance': val[1]})
        df = pd.DataFrame(recommend_frame, index=range(1, n_items_to_recommend+1))
        return df
    else:
        return "Item not found in the dataset"

# Test the recommendation function with a random item from the training set
random_item = random.choice(pivot_train_data.index)
print("Random item from training set:", random_item)
print("Items recommended for item:", get_item_recommendation(random_item))



Random item from training set: BIRDS MOBILE VINTAGE DESIGN
Items recommended for item:                            ItemName  Distance
1         LETTER "G" BLING KEY RING  0.776839
2      CURIOUS  IMAGES NOTEBOOK SET  0.775009
3                 TOOL BOX SOFT TOY  0.774126
4         BLACK EAR MUFF HEADPHONES  0.773426
5          RETROSPOT CANDLE  MEDIUM  0.771625
6       CABIN BAG VINTAGE RETROSPOT  0.767239
7   WOODLAND  HEIGHT CHART STICKERS  0.757591
8       SET OF 4 CAROUSEL PLACEMATS  0.752594
9      BLOSSOM  IMAGES NOTEBOOK SET  0.752072
10            MOBILE VINTAGE HEARTS  0.634253


In [18]:
# Initializing counters for precision, recall, and f1 score
count_precision_train = 0
count_recall_train = 0
count_f1_train = 0


train_customers = train_data['CustomerID'].unique()


for customer_id in train_customers:

    train_items = train_data[train_data['CustomerID'] == customer_id]['Itemname']


    if not train_items.empty:

        recommended_items = get_item_recommendation(train_items.iloc[0])


        count_relevant_recommended = sum(recommended_items['ItemName'].isin(train_items))


        precision_train = count_relevant_recommended / len(recommended_items)
        recall_train = count_relevant_recommended / len(train_items)


        if precision_train + recall_train != 0:
            f1_train = 2 * (precision_train * recall_train) / (precision_train + recall_train)
            count_f1_train += f1_train

        count_precision_train += precision_train
        count_recall_train += recall_train

# Computing the average precision, recall, and f1 score over all customers in the training set
precision_train = count_precision_train / len(train_customers)
recall_train = count_recall_train / len(train_customers)
f1_train = count_f1_train / len(train_customers)


print("Training Precision:", precision_train)
print("Training Recall:", recall_train)
print("Training F1 Score:", f1_train)


Training Precision: 0.14037880867417182
Training Recall: 0.06281505935758305
Training F1 Score: 0.06603643511529113


In [19]:

# Test the recommendation function with a random customer from the test set
random_customer_id = random.choice(test_data['CustomerID'].unique())
test_items = test_data[test_data['CustomerID'] == random_customer_id]['Itemname']
print("Items bought by customer:", test_items)
print("Items recommended to customer:", get_item_recommendation(test_items.iloc[0]))

Items bought by customer: 21668                 DOORMAT RED RETROSPOT
21669                       DOORMAT AIRMAIL
21670        BOX OF VINTAGE ALPHABET BLOCKS
21671                    RABBIT NIGHT LIGHT
21672               EDWARDIAN PARASOL BLACK
21673           DOORMAT WELCOME TO OUR HOME
21674        MINI LIGHTS WOODLAND MUSHROOMS
21675       VINTAGE DOILY DELUXE SEWING KIT
21676    SET/3 RED GINGHAM ROSE STORAGE BOX
21677                  JUMBO BAG STRAWBERRY
Name: Itemname, dtype: object
Items recommended to customer:                              ItemName  Distance
1   DOORMAT UNION JACK GUNS AND ROSES  0.684215
2         DOORMAT WELCOME TO OUR HOME  0.675346
3       DOORMAT KEEP CALM AND COME IN  0.664926
4       DOORMAT VINTAGE LEAVES DESIGN  0.660429
5          DOORMAT MULTICOLOUR STRIPE  0.648771
6        DOORMAT HOME SWEET HOME BLUE  0.637899
7                 DOORMAT NEW ENGLAND  0.607646
8                  DOORMAT UNION FLAG  0.596539
9                DOORMAT ENGLISH ROSE  0.

In [20]:
# Initialize counters for precision, recall, and f1 score
count_precision = 0
count_recall = 0
count_f1 = 0


test_customers = test_data['CustomerID'].unique()


for customer_id in test_customers:

    test_items = test_data[test_data['CustomerID'] == customer_id]['Itemname']


    if not test_items.empty:

        recommended_items = get_item_recommendation(test_items.iloc[0])


        if isinstance(recommended_items, pd.DataFrame):
            count_relevant_recommended = sum(recommended_items['ItemName'].isin(test_items))


            precision = count_relevant_recommended / len(recommended_items)
            recall = count_relevant_recommended / len(test_items)


            if precision + recall != 0:
                f1 = 2 * (precision * recall) / (precision + recall)
                count_f1 += f1

            count_precision += precision
            count_recall += recall

# Computing the average precision, recall, and f1 score over all customers in the test set
precision = count_precision / len(test_customers)
recall = count_recall / len(test_customers)
f1 = count_f1 / len(test_customers)


print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Precision: 0.09723844928305786
Recall: 0.06402033760293083
F1 Score: 0.06090190897140408


### 5.Reflection


I have learned and gained knowledge on modeling and tuning the collaborative filter recommendation model.

### 6. References

### https://github.com/So-ham/Movie-Recommendation-System/blob/main/movie-recommendation.ipynb